참고자료  
- https://nlp.seas.harvard.edu/2018/04/03/attention.html  
- https://colab.research.google.com/github/metamath1/ml-simple-works/blob/master/transformer/annotated_transformer.ipynb#scrollTo=lKThOQWr0DGO

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy

In [2]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model) # look-up table
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, droput, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=droput)

        # pos/10000^(2i/d_model) = pos * exp(-((2i)/d_model)*log(10000))
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqeeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        pe_val = self.pe[:, :x.size(1)]
        pe_val.requires_grad = False

        x += pe_val
        return self.dropout(x)

In [4]:
def attention(query, key, value, mask=None, dropout=None):
    # query: (nbatches, h, n_seq, d_k)
    # key:   (nbatches, h, n_seq, d_k)
    # value: (nbatcehs, h, n_seq, d_v)

    d_k = query.size(-1)

    # scores: (nbatches, h, n_seq, n_seq)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask==0, -1e9)

    p_attention = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attention = dropout(p_attention)

    # p_attention                     : (nbatches, h, n_seq, nseq)
    # torch.matmul(p_attention, value): (nbatches, h, n_seq, d_v)
    return torch.matmul(p_attention, value), p_attention

In [5]:
def clones(module, N):
    # Produces N identical layers
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [6]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h # assume d_k == d_v
        self.h = h

        # Wq, Wk, Wv, Wo
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        
    def forward(self, query, key, value, mask=None):
        # q, k, v: (n_seq, d_model)
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        for l, x in zip(self.linears, (query, key, value)):
            # q, k, v -> (nbatches, h, n_seq, d_k)
            query, key, value = [l(x).view(nbatches, -1, self.h, self._k).transpose(1,2)]

        # x: (nbatches, h, n_seq, d_v)
        # self.attention: (nbatches, h, n_seq, n_seq)
        x, self.attention = attention(query, key, value, mask=mask, dropout=self.dropout)
        
        # x: (nbatches, n_seq, h*d_k)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)

        # matmul x and Wo -> (nbatches, n_seq, d_model)
        return self.linears[-1](x)

In [7]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, unbiased=False, keepdim=True)
        
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [8]:
class SublayerConnection(nn.Module):
    # Residual connection followed by a layer norm.
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        # (attention or ff)-[dropout]-Add and Norm
        return self.norm(x + self.dropout(sublayer(x)))

In [9]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        
        # self_attn: MultiHeadedAttention
        self.self_attn = self_attn

        # feed_forward: PositionwiseFeedForward
        self.feed_forward = feed_forward 
        
        # 2 sublayers
        self.sublayer = clones(SublayerConnection(size, dropout), 2) 
        self.size = size # d_model

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [10]:
class Encoder(nn.Module):
    # stack of N EncoderLayer(N=6)
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N) 
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):        
        for layer in self.layers: 
            x = layer(x, mask)
        return x

In [11]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [12]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        # self_attn: MultiHeadedAttention
        # src_attn: MultiHeadedAttention

        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn # cross attention
        self.feed_forward = feed_forward # positional ff
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        
        return self.sublayer[2](x, self.feed_forward)

In [13]:
class Decoder(nn.Module):
    # stack of N DecoderLayer(N=6)
    def __init__(self, layer, N):
        # layer: DecoderLayer
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return x

In [14]:
class Generator(nn.Module):
    # linear projection + softmax
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [15]:
class EncoderDecoder(nn.Module):
    # standard Encoder-Decoder architecture
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(
            self.encode(src, src_mask), src_mask,
            tgt, tgt_mask
        )
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [16]:
def tansformer(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attention = MultiHeadedAttention(h, d_model)
    feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)

    model = EncoderDecoder(      
        # Input & Output Embeddings
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        
        # Encoder & Decoder
        Encoder(EncoderLayer(d_model, c(attention), c(feed_forward), dropout), N),
        Decoder(DecoderLayer(d_model, c(attention), c(feed_forward), dropout), N),

        Generator(d_model, tgt_vocab)
    )

    return model